In [1]:
from meerkat.base import time
from meerkat import atlas

In [2]:
from importlib import reload

In [3]:
#reload(atlas)
dev = atlas.Conductivity(bus_n=1, output='csv')  # csv is default

In [4]:
# this fails to write JSON correctly and dies
dev.write(description='test_csv', n=5)

In [5]:
dev.writer.path

'2019_04_15_18_16_58_data.csv'

In [6]:
!head $dev.writer.path

#!{"accuracy": null, "bytes": null, "case_sensitive_header": false, "comment": "#", "delimiter": ",", "description": null, "device": {"accuracy": null, "active": null, "bus": "<meerkat.i2c_pi.WrapI2C object at 0x70f81ef0>", "calibration_date": null, "description": "", "dtype": null, "error": null, "manufacturer": "Atlas Scientific", "name": "Atlas_Base", "precision": "Varies", "state": null, "urls": "www.atlas-scientific.com", "version_hw": "1.0", "version_sw": "1.0"}, "double_quote": true, "dtypes": null, "encoding": "utf-8", "escape_char": "\\", "format": null, "hash": null, "header": ["description", "sample_n", "conductivity", "total_dissolved_solids", "salinity", "specific_gravity"], "licenses": null, "line_terminator": "\n", "media_type": "text/csv", "name": "Atlas_Conductivity", "null_sequence": "NA", "path": "2019_04_15_18_16_58_data.csv", "precision": null, "quote_char": "\"", "schema": null, "shebang": true, "skip_initial_space": true, "skip_lines": 1, "sources": null, "standa

In [7]:
# Dry and two point calibration
dev.cal_set_dry()
time.sleep(1)
dev.cal_set_low(2000)
time.sleep(1)
dev.cal_set_high(10000)
time.sleep(1)
dev.cal_get()

0

In [8]:
# clear calibration
time.sleep(1)
dev.cal_clear()
time.sleep(1)
dev.cal_get()

0

In [9]:
# probe type
time.sleep(0.5)
dev.set_probe_type(10.0)
time.sleep(0.5)
dev.get_probe_type()

10.0

In [10]:
# temperature compensation
time.sleep(0.5)
dev.temp_set(24.2)
time.sleep(0.5)
dev.temp_get()

24.19

In [11]:
# device information: device type, firmware version
time.sleep(0.5)
dev.info()

('EC', '1.5')

In [12]:
# status of device power: restart code, input voltage Vcc
time.sleep(0.5)
dev.status()

('P', 3.08)

In [13]:
# measurement output formats
dev.output_get()

['EC', 'TDS', 'S', 'SG']

In [14]:
# measure conductivity
time.sleep(0.5)
dev.measure()

[0.0, 0.0, 0.0, 1.0]

In [15]:
dev.get()

['no_description', 0, 0.0, 0.0, 0.0, 1.0]

In [16]:
dev.get('test_1')

['test_1', 0, 0.0, 0.0, 0.0, 1.0]

In [17]:
dev.get('test_2', 5)

[['test_2', 0, 0.0, 0.0, 0.0, 1.0],
 ['test_2', 1, 0.0, 0.0, 0.0, 1.0],
 ['test_2', 2, 0.0, 0.0, 0.0, 1.0],
 ['test_2', 3, 0.0, 0.0, 0.0, 1.0],
 ['test_2', 4, 0.0, 0.0, 0.0, 1.0]]

In [18]:
dev.writer.header

['description',
 'sample_n',
 'conductivity',
 'total_dissolved_solids',
 'salinity',
 'specific_gravity']

In [19]:
dev = atlas.Conductivity(bus_n=1, output='json')  # csv is default

In [20]:
dev.writer.metadata_interval = 3

In [21]:
dev.write(description='test_json', n=7)

In [22]:
dev.writer.path

'2019_04_15_18_17_30_JSON_data.txt'

In [23]:
!head $dev.writer.path

{"std_time_ms": "2019-04-15 18:17:30.331643", "data": ["test_json", 0, 0.0, 0.0, 0.0, 1.0]}
{"std_time_ms": "2019-04-15 18:17:32.492241", "data": ["test_json", 1, 0.0, 0.0, 0.0, 1.0]}
{"metadata": "{\"accuracy\": null, \"bytes\": null, \"comment\": \"#\", \"description\": null, \"device\": {\"accuracy\": null, \"active\": null, \"bus\": \"<meerkat.i2c_pi.WrapI2C object at 0x70faa2f0>\", \"calibration_date\": null, \"description\": \"\", \"dtype\": null, \"error\": null, \"manufacturer\": \"Atlas Scientific\", \"name\": \"Atlas_Base\", \"precision\": \"Varies\", \"state\": null, \"urls\": \"www.atlas-scientific.com\", \"version_hw\": \"1.0\", \"version_sw\": \"1.0\"}, \"double_quote\": true, \"dtypes\": null, \"encoding\": \"utf-8\", \"escape_char\": \"\\\\\", \"format\": null, \"hash\": null, \"header\": [\"description\", \"sample_n\", \"conductivity\", \"total_dissolved_solids\", \"salinity\", \"specific_gravity\"], \"licenses\": null, \"line_terminator\": \"\\n\", \"media_type\": \"t